# Milvus Vector Database of Blue Core Triples

In [1]:
import json
import pathlib
import random

import httpx

from pymilvus import model, MilvusClient

from bluecore_models.utils.graph import load_jsonld

/Users/jpnelson/50-59 Presentations, Training, and Education/50.17 DCMI 2025 - Shared Open Metadata as Critical AI Infrastructure/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
data_path = pathlib.Path("../data")
blue_core_url = "https://dev.bcld.info"

In [3]:
milvus_client = MilvusClient("dcmi_demo.db")
milvus_client.create_collection(
    auto_id=True,
    collection_name="instances",
    dimension=768,
)
milvus_client.create_collection(
    auto_id=True,
    collection_name="works",
    dimension=768,
)
embedding_func = model.DefaultEmbeddingFunction()

/Users/jpnelson/50-59 Presentations, Training, and Education/50.17 DCMI 2025 - Shared Open Metadata as Critical AI Infrastructure/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [4]:
with (data_path / "random-100-works.jsonl").open() as fo:
    works = []
    for row in fo.readlines():
        works.append(json.loads(row))

In [5]:
with (data_path / "random-instances.jsonl").open() as fo:
    instances = []
    for row in fo.readlines():
        instances.append(json.loads(row))

In [6]:
def create_embedding(entity, type_of):
    graph = load_jsonld(entity['data'])
    resource_uri = entity['uri']
    skolemized_graph = graph.skolemize(basepath=f"{resource_uri}#")
    triples = [
        line.strip(".")
        for line in skolemized_graph.serialize(format="nt").splitlines()
    ]
    triple_vectors = embedding_func.encode_documents(triples)
    embeddings_data = []
    for i,vector in enumerate(triple_vectors):
        embeddings_data.append({
            "vector": vector,
            "text": triples[i],
            "version": entity['version'],
            "uri": resource_uri
        })
    result = milvus_client.insert(collection_name=type_of, data=embeddings_data)

In [7]:
for work in works:
    create_embedding(work, "works")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<built-in method fromisoformat of type object at 0x1031fb400>
Traceback (most recent call last):
  File "/Users/jpnelson/50-59 Presentations, Training, and Education/50.17 DCMI 2025 - Shared Open Metadata as Critical AI Infrastructure/.venv/lib/python3.12/site-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: Invalid isoformat string: '2025-04-10T::'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<built-in method fromisoformat of type object at 0x1031fb400>
Traceback (most recent call last):
  File "/Users/jpnelson/50-59 Presentations, Training, and Education/50.17 DCMI 2025 - Shared Open Metadata as Critical AI Infrastructure/.venv/lib/python3.12/site-packages/rdflib/term.py", line 2163, in _castLexicalToP

In [8]:
for instance in instances:
    create_embedding(instance, "instances")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<built-in method fromisoformat of type object at 0x1031fb400>
Traceback (most recent call last):
  File "/Users/jpnelson/50-59 Presentations, Training, and Education/50.17 DCMI 2025 - Shared Open Metadata as Critical AI Infrastructure/.venv/lib/python3.12/site-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: Invalid isoformat string: '2025-04-10T::'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<built-in method fromisoformat of type object at 0x1031fb400>
Traceback (most recent call last):
  File "/Users/jpnelson/50-59 Presentations, Training, and Education/50.17 DCMI 2025 - Shared Open Metadata as Critical AI Infrastructure/.venv/lib/python3.12/site-packages/rdflib/term.py", line 2163, in _castLexicalToP